In [1]:
import pandas as pd
from marginal.main import estimar_marginal
from thing import condicional


df = pd.read_csv("datasets/training_dataset1.csv")
df

14
12
<class 'tuple'>
<class 'str'>
<class 'tuple'>
<class 'str'>
<class 'tuple'>
<class 'str'>
<class 'tuple'>
<class 'str'>
<class 'tuple'>
<class 'str'>
<class 'tuple'>
<class 'str'>
<class 'tuple'>
<class 'str'>
<class 'tuple'>
<class 'str'>
<class 'tuple'>
<class 'str'>
<class 'tuple'>
<class 'str'>
<class 'tuple'>
<class 'str'>
<class 'tuple'>
<class 'str'>
{('overcast', 'yes', False): 0.33333333333333337, ('overcast', 'yes', True): 0.5, ('overcast', 'no', False): 0.19999999999999998, ('overcast', 'no', True): 0.16666666666666669, ('rainy', 'yes', False): 0.44444444444444453, ('rainy', 'yes', True): 0.16666666666666669, ('rainy', 'no', False): 0.19999999999999998, ('rainy', 'no', True): 0.5, ('sunny', 'yes', False): 0.22222222222222227, ('sunny', 'yes', True): 0.33333333333333337, ('sunny', 'no', False): 0.6, ('sunny', 'no', True): 0.33333333333333337}


,A,B,C
0,0,0,0
1,2,1,1
2,2,1,0
3,0,1,0
4,0,0,1
5,1,1,1
6,1,1,1
7,2,0,0
8,0,1,0
9,1,1,1


In [2]:
def foo(df,colNames, values):
    l = []
    for i in range(len(colNames)):
        l.append(df[colNames[i]] == values[i])
        
#     return df.loc()
#     print(l)
    
    x = l[0]
    for i in range(1,len(l)):
        x = x&l[i]
        
    return df.loc[x]

In [3]:
def findVariables(df, col):
    l = []
    for elem in df[col]:
    	if elem not in l:
    		l.append(elem)
    return l

In [4]:
def getCardinality(df, col):
    return len(findVariables(df,col))

In [5]:
# Desde aqui importa <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

m = [[0,1,0],
     [0,0,1],
     [0,0,0]]

def getParents(g,df):
    fathers = []
    res = []
    colNames = list(df.columns)
    
    for col in range(len(g)):
        aux = []
        for row in range(len(g)):
            if(m[col][row] == 1):
                aux.append(colNames[row])
        fathers.append(aux)

    for i in range(len(g)):
        yield [colNames[i],fathers[i]]
    
    #return res

for i in getParents(m,df):
    print(i)

['A', ['B']]
['B', ['C']]
['C', []]


In [6]:
class rb:
    def __init__(self,g, df,a):
        self.dag = g
        self.margs = [] # => tienes que correr calc() para llenarlo
        self.conds = []
        self.alpha = a
        self.df = df
    
    def calc(self):
        for elem in getParents(self.dag,self.df):
            if(len(elem[1])== 0):
                self.margs.append([elem,estimar_marginal(df,elem[0],self.alpha)])
            else:
                self.conds.append([elem,condicional(df,elem[0],elem[1],self.alpha)])

In [66]:
def inferencia(df,q,colNames, values, rb):
    mgs = []
    cds = []
    accum = 1
    final = {}
    
    for i in rb.conds:
        idxs = []
        e = [i[0][0]] + [x for x in i[0][1]]
        flag = 0
        vals = []
        
        if(set(e).issubset(set(colNames))):
            flag = 1
            
            
        if flag == 1:    
            for elem in e:
                idxs.append(colNames.index(elem))
            
        else:
            uniques = list(df[q].unique())
            tmp = []
#             print("e->>>",e)
            tmp_e = e.copy()
            tmp_e.remove(q)
#             print("tmp_e->>>>",tmp_e)
            
            for elem in tmp_e:
                idxs.append(colNames.index(elem))
            
            for idx in idxs:
                tmp.append(values[idx])
            
            expectedResults = [tuple([u]+tmp) for u in uniques]
                
            
#             print("EXPECTED RESULTS",expectedResults)

            for c in expectedResults:
                final[c] = i[1][c]
            
            
            newFinal = {}
            for f in final:
                newFinal[f[0]] = final[f]
            
            del final
                
            dictValues = i[1].values()
            continue
        
        
        for idx in idxs:
            vals.append(values[idx])
        
        cds.append(i[1][tuple(vals)])
        
        
    for i in rb.margs:        
        var = i[0][0]
        idx = colNames.index(var)
        
        mgs.append(i[1][values[idx]])

    
    for i in range(len(mgs)):
        accum *= mgs[i]
        
#     print("MGS",mgs)    
    for i in range(len(cds)):
        accum *= cds[i]
#     print("CDS",cds)
#     print("FINAL ANTES",newFinal)
    for i in newFinal:
        newFinal[i] *= accum
    
#     print(accum)
    return newFinal

In [67]:
x = rb(m,df,1)
x.calc()
x.margs
# x.conds

10
6
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>
10
4
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>
<class 'tuple'>
<class 'int'>


[[['C', []], {0: 0.5, 1: 0.5}]]

In [68]:
print("Solucion",inferencia(df,"A",["B","C"],[1,0],x))

Solucion {0: 0.0857142857142857, 2: 0.0857142857142857, 1: 0.11428571428571428}
